In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [4]:
# Specify the path to JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'propertyassessment'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)


property_assessment_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.shape)
    property_assessment_df = df.copy()

fy2024-property-assessment-data_1_5_2024.csv


<ipython-input-4-721f973491e2>:30: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


(182242, 65)


In [5]:
# check the columns name
property_assessment_df.columns

Index(['PID', 'CM_ID', 'GIS_ID', 'ST_NUM', 'ST_NAME', 'UNIT_NUM', 'CITY',
       'ZIP_CODE', 'BLDG_SEQ', 'NUM_BLDGS', 'LUC', 'LU', 'LU_DESC',
       'BLDG_TYPE', 'OWN_OCC', 'OWNER', 'MAIL_ADDRESSEE',
       'MAIL_STREET_ADDRESS', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP_CODE',
       'RES_FLOOR', 'CD_FLOOR', 'RES_UNITS', 'COM_UNITS', 'RC_UNITS',
       'LAND_SF', 'GROSS_AREA', 'LIVING_AREA', 'LAND_VALUE', 'BLDG_VALUE',
       'SFYI_VALUE', 'TOTAL_VALUE', ' GROSS_TAX ', 'YR_BUILT', 'YR_REMODEL',
       'STRUCTURE_CLASS', 'ROOF_STRUCTURE', 'ROOF_COVER', 'INT_WALL',
       'EXT_FNISHED', 'INT_COND', 'EXT_COND', 'OVERALL_COND', 'BED_RMS',
       'FULL_BTH', 'HLF_BTH', 'KITCHENS', 'TT_RMS', 'BDRM_COND',
       'BTHRM_STYLE1', 'BTHRM_STYLE2', 'BTHRM_STYLE3', 'KITCHEN_TYPE',
       'KITCHEN_STYLE1', 'KITCHEN_STYLE2', 'KITCHEN_STYLE3', 'HEAT_TYPE',
       'HEAT_SYSTEM', 'AC_TYPE', 'FIREPLACES', 'ORIENTATION', 'NUM_PARKING',
       'PROP_VIEW', 'CORNER_UNIT'],
      dtype='object')

In [6]:
property_assessment_df.head()

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIP_CODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_SYSTEM,AC_TYPE,FIREPLACES,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,NaN,100001000,104.0,PUTNAM ST,NaN,EAST BOSTON,2128.0,1,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.0,NaN,3.0,A - Average,NaN
1,100002000,NaN,100002000,197.0,Lexington ST,NaN,EAST BOSTON,2128.0,1,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0.0,NaN,0.0,A - Average,NaN
2,100003000,NaN,100003000,199.0,Lexington ST,NaN,EAST BOSTON,2128.0,1,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0.0,NaN,0.0,A - Average,NaN
3,100004000,NaN,100004000,201.0,Lexington ST,NaN,EAST BOSTON,2128.0,1,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.0,NaN,0.0,A - Average,NaN
4,100005000,NaN,100005000,203.0,Lexington ST,NaN,EAST BOSTON,2128.0,1,1,...,N - No Remodeling,NaN,W - Ht Water/Steam,NaN,N - None,0.0,NaN,0.0,A - Average,NaN


In [7]:
# Check for null
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182242 entries, 0 to 182241
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PID                  182242 non-null  int64  
 1   CM_ID                93291 non-null   float64
 2   GIS_ID               182242 non-null  int64  
 3   ST_NUM               172879 non-null  float64
 4   ST_NAME              182242 non-null  object 
 5   UNIT_NUM             82613 non-null   object 
 6   CITY                 182239 non-null  object 
 7   ZIP_CODE             182239 non-null  float64
 8   BLDG_SEQ             182242 non-null  int64  
 9   NUM_BLDGS            182242 non-null  int64  
 10  LUC                  182242 non-null  int64  
 11  LU                   182242 non-null  object 
 12  LU_DESC              182242 non-null  object 
 13  BLDG_TYPE            179626 non-null  object 
 14  OWN_OCC              182242 non-null  object 
 15  OWNER            

In [8]:
# Get rid of columns that contain over 60% null values
columns_to_drop = ['CM_ID', 'UNIT_NUM','MAIL_ADDRESSEE','CD_FLOOR','RES_UNITS',
                   'COM_UNITS','RC_UNITS','STRUCTURE_CLASS','BDRM_COND',
                   'BTHRM_STYLE2','BTHRM_STYLE3','KITCHEN_STYLE2','KITCHEN_STYLE3',
                   'HEAT_SYSTEM','ORIENTATION','CORNER_UNIT']
df.drop(columns=columns_to_drop, inplace=True)

In [9]:
# Clean null values
clean_data= df.dropna(inplace=True)

In [10]:
# # Check for null values again
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68853 entries, 1 to 182239
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PID                  68853 non-null  int64  
 1   GIS_ID               68853 non-null  int64  
 2   ST_NUM               68853 non-null  float64
 3   ST_NAME              68853 non-null  object 
 4   CITY                 68853 non-null  object 
 5   ZIP_CODE             68853 non-null  float64
 6   BLDG_SEQ             68853 non-null  int64  
 7   NUM_BLDGS            68853 non-null  int64  
 8   LUC                  68853 non-null  int64  
 9   LU                   68853 non-null  object 
 10  LU_DESC              68853 non-null  object 
 11  BLDG_TYPE            68853 non-null  object 
 12  OWN_OCC              68853 non-null  object 
 13  OWNER                68853 non-null  object 
 14  MAIL_STREET_ADDRESS  68853 non-null  object 
 15  MAIL_CITY            68853 non-null  obj

In [13]:
df.head()

,PID,GIS_ID,ST_NUM,ST_NAME,CITY,ZIP_CODE,BLDG_SEQ,NUM_BLDGS,LUC,LU,...,KITCHENS,TT_RMS,BTHRM_STYLE1,KITCHEN_TYPE,KITCHEN_STYLE1,HEAT_TYPE,AC_TYPE,FIREPLACES,NUM_PARKING,PROP_VIEW
1,100002000,100002000,197.0,Lexington ST,EAST BOSTON,2128.0,1,1,105,R3,...,3.0,9.0,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,F - Forced Hot Air,C - Central AC,0.0,0.0,A - Average
2,100003000,100003000,199.0,Lexington ST,EAST BOSTON,2128.0,1,1,105,R3,...,3.0,13.0,M - Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Space Heat,N - None,0.0,0.0,A - Average
3,100004000,100004000,201.0,Lexington ST,EAST BOSTON,2128.0,1,1,105,R3,...,3.0,11.0,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,W - Ht Water/Steam,N - None,0.0,0.0,A - Average
4,100005000,100005000,203.0,Lexington ST,EAST BOSTON,2128.0,1,1,104,R2,...,2.0,13.0,N - No Remodeling,2F - 2 Full Eat In Kitchens,N - No Remodeling,W - Ht Water/Steam,N - None,0.0,0.0,A - Average
5,100006000,100006000,205.0,Lexington ST,EAST BOSTON,2128.0,1,1,105,R3,...,3.0,20.0,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,E - Electric,N - None,0.0,0.0,A - Average


In [14]:
csv_bytes = df.to_csv(index=False).encode()
blob_name = "clean_data_no_nulls.csv"

In [15]:
# Upload the CSV bytes to Azure
blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob(csv_bytes, overwrite=True)

{'etag': '"0x8DC6574014E686A"',
 'last_modified': datetime.datetime(2024, 4, 25, 22, 6, 49, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'5:}`Ca/\x8d^v7\xf3\xaaj\x8f\x81'),
 'client_request_id': '1c7e6264-0350-11ef-a982-0242ac1c000c',
 'request_id': '1fed5bf3-f01e-0067-175c-9752f1000000',
 'version': '2023-11-03',
 'version_id': None,
 'date': datetime.datetime(2024, 4, 25, 22, 6, 48, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}